# FHN equations

$$
\begin{align}
\partial_t u &= D_u \Delta u + [u(1-u)(u-a) -v  ]/\epsilon\\
\partial_t v &= D_v \Delta v + \epsilon (u - \gamma v)
\end{align}
$$

$$
(a, \epsilon, \gamma, D_u, D_v) 
= (0.25, 0.05, 3.0, 4.0 \times 10^{-5}, 1.0\times 10^{-7})
$$

## Importing modules 

In [1]:
import sys
sys.path.insert(0, '..')

In [2]:
%matplotlib qt5

In [3]:
import numpy as np
import equations as eq
from scipy.integrate import odeint
from scipy.fftpack import rfft, irfft

import h5py

In [4]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

## Setting Parameters 

In [5]:
NW = 100
fhn01 = eq.FHN(NW)
args = fhn01.getParamDefault()

In [6]:
N2 = fhn01.N2
J = fhn01.J
x = fhn01.get_x()

## Setting initial data 

In [7]:
uv0 = np.zeros(2*N2)
u0,v0 = uv0.reshape((2,N2))
U0 = np.zeros(J) + 0.4*np.exp(-800.0*(x-0.4)**2)
V0 = np.zeros(J) + 5.0e-2*np.exp(-800.0*(x-0.35)**2)

u0[:] = rfft(U0)[:N2]
v0[:] = rfft(V0)[:N2]

## Computation 

In [8]:
trange = np.linspace(0.0, 100.0, 5001)
dt = 1.0e-4

with h5py.File('fhn.hdf5', 'w') as fh:
    fhn01.mkInitDataSet(uv0, fh, dt)    
    fhn01.evolve(fh,trange,tuple(args))
    data = fh['u'][()]

In [9]:
up = fhn01.mkPhysData(
        fhn01.reshapeTS(data, len(trange)))

C:\Users\rikota\Anaconda3\lib\site-packages\scipy\fftpack\basic.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  z[index] = x


## Visualization 

In [10]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)

ax.set_ylim((-0.5, 1.5))
ax.set_xlim((-0.1, 1.1))
ax.grid(True)
lines = plt.plot(x, [np.nan]*len(x), x, [np.nan]*len(x));

In [11]:
def init():
    for l in lines:
        l.set_ydata([np.nan] * len(x))
    return lines

def animate(i):
    for j, l in enumerate(lines):
        l.set_ydata(up[i,j,:])
    return lines

In [12]:
ani = animation.FuncAnimation(
    fig, animate, frames=len(trange),
    init_func=init, interval=1, blit=True,
    repeat=True)

plt.show()